In [1]:
# import opendatasets as od
# import os
import pandas as pd
import numpy as np
import copy
import itertools


In [10]:
base_rest_df = pd.read_csv('../restaurant_price_base.csv', sep=';')
base_rest_df.head()

,restaurant_link,country,region,province,city,avg_rating,total_reviews_count,price_level,price_min,price_max
0,g10001637-d10002227,France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,4.0,36.0,€,14.0,29.0
1,g10001637-d14975787,France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,4.0,5.0,€,8.0,17.0
2,g10002858-d4586832,France,Centre-Val de Loire,Berry,Rivarennes,5.0,13.0,€,10.0,35.0
3,g10002986-d3510044,France,Nouvelle-Aquitaine,Correze,Lacelle,4.0,34.0,€,12.0,26.0
4,g10022428-d9767191,France,Occitanie,Aveyron,Saint-Laurent-de-Levezou,4.5,11.0,€€-€€€,12.0,30.0


In [11]:
base_rest_df.shape

(304327, 10)

In [12]:
base_rest_df.columns

Index(['restaurant_link', 'country', 'region', 'province', 'city',
       'avg_rating', 'total_reviews_count', 'price_level', 'price_min',
       'price_max'],
      dtype='object')

In [14]:
price_df = base_rest_df[['restaurant_link', 'country', 'region', 'province', 'city', 
                         'price_level', 
                         'avg_rating', 'total_reviews_count', 
                         'price_min', 'price_max']] #[base_rest_df['price_min'].notna()]

price_df.head()

,restaurant_link,country,region,province,city,price_level,avg_rating,total_reviews_count,price_min,price_max
0,g10001637-d10002227,France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,€,4.0,36.0,14.0,29.0
1,g10001637-d14975787,France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,€,4.0,5.0,8.0,17.0
2,g10002858-d4586832,France,Centre-Val de Loire,Berry,Rivarennes,€,5.0,13.0,10.0,35.0
3,g10002986-d3510044,France,Nouvelle-Aquitaine,Correze,Lacelle,€,4.0,34.0,12.0,26.0
4,g10022428-d9767191,France,Occitanie,Aveyron,Saint-Laurent-de-Levezou,€€-€€€,4.5,11.0,12.0,30.0


In [15]:
price_df.shape

(304327, 10)

In [20]:
price_df_short = price_df[['price_min', 'price_max']]

In [17]:
from pyod.models.iforest import IForest

In [21]:
iforest = IForest()

labels = iforest.fit_predict(price_df_short)
print(labels)

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict is deprecated
  warnings.warn(msg, category=FutureWarning)


[0 0 0 ... 0 0 0]


In [23]:
outliers = price_df_short[labels == 1]
print(outliers.shape)

(30429, 2)


## Hyperparameter tuning

In [28]:
from itertools import product

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [27]:
def evaluate_outlier_classifier(model, data):
    # Get labels
    labels = model.fit_predict(data)
    # Return inliers
    return data[labels == 0]

def evaluate_regressor(inliers):
    X = inliers.drop("sales", axis=1)
    y = inliers[['sales']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    preds = lr.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    return round(rmse, 3)

In [25]:
contaminations = [0.05, 0.1, 0.2, 0.3]
estimators = [100, 200, 300]
max_samples = [0.6, 0.8, 1]
max_features = [1]

scores = dict()

In [26]:
for c, e, s, f in product(contaminations, estimators, max_samples, max_features):
    # Instantiate an IForest
    iforest = IForest(contamination=c, n_estimators=e, max_samples=s, max_features=f)
    # Get the inliers with the current IForest
    inliers = evaluate_outlier_classifier(iforest, price_df_short)
    # Calculate and store RMSE into scores
    scores[(c, e, s, f)] = evaluate_regressor(inliers)

NameError: name 'evaluate_outlier_classifier' is not defined

In [ ]:
 # Faster compuation with n_jobs=-1
iforest = IForest(n_estimators=1000, n_jobs=-1)
iforest.fit(big_mart)

In [ ]:
iforest = IForest(contamination=0.05)

In [ ]:
iforest = IForest(n_estimators=1000)
iforest.fit(airbnb_df)

In [ ]:
iforest = IForest(n_estimators=200, max_samples=0.6, max_features=0.9)
iforest.fit(airbnb_df)

In [ ]:
iforest = IForest(contamination=0.05, max_features=0.5, random_state=1)
iforest = iforest.fit(airbnb_df)
labels = iforest.labels_
print(labels)